In [ ]:
import jarvis
import pandas as pd
import numpy as np

Waiting for redis server at 127.0.0.1:15055 to respond...
Waiting for redis server at 127.0.0.1:29309 to respond...
Starting local scheduler with 4 CPUs, 0 GPUs

View the web UI at http://localhost:8889/notebooks/ray_ui12288.ipynb?token=56236bd1362b4c2616cef287b4bffee95a90cd6ba0f158dd



# Where do I start? Summarize current and past experiments

In [ ]:
summary = jarvis.versionSummaries('twitter_demo')

In [ ]:
summary

# What is the best experiment?
Here, we measure _goodness_ of an experiment by its max model accuracy, over all trials.

In [ ]:
summary_stat = summary.groupby('__commitHash__')['model_accuracy'].max()
summary_stat

In [ ]:
best_commit_hash = summary_stat.idxmax()
best_commit_hash[0:5] + '...'

# What is the best trial, given the best experiment?

In [ ]:
best_index = summary.loc[summary['__commitHash__'] == best_commit_hash]['model_accuracy'].idxmax()
best_trial = summary.iloc[best_index]['__trialNum__']
best_trial

# So the former experiment is better than the latter. What changed?

In [ ]:
versions = summary.__commitHash__.unique()
most_recent, least_recent = versions

In [ ]:
jarvis.diffExperimentVersions('twitter_demo', least_recent, most_recent)

# Let's get the best model yet!
From the best trial, from the best experiment

In [ ]:
best_model = jarvis.checkoutArtifact('twitter_demo', best_trial, best_commit_hash, 'model.pkl')

# Here's the best model in action

In [ ]:
country_dict = best_model['country_dict']
classifier = best_model['classifier']
vectorizer = best_model['vectorizer']

code_dict = {}

for kee in country_dict:
    code_dict[country_dict[kee]] = kee

while True:
    tweet = input("What's on your mind? ")
    if tweet == 'exit':
        break
    tweet_vec = vectorizer.transform(np.array([tweet,]))
    country_id = classifier.predict(tweet_vec)
    print("Predicted country of origin: {}\n".format(code_dict[country_id[0]]))